In [49]:
import xgboost as xgb

In [50]:
import numpy as np
import pandas as pd


In [51]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score,roc_auc_score,make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import StandardScaler


In [52]:
df=pd.read_csv('train.csv')

In [53]:
df.head()

,customer_id,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
0,CST_115179,ita Bose,46,F,N,Y,0.0,107934.04,612.0,Unknown,1.0,1.0,33070.28,18690.93,73,544,2,1,1
1,CST_121920,Alper Jonathan,29,M,N,Y,0.0,109862.62,2771.0,Laborers,2.0,0.0,15329.53,37745.19,52,857,0,0,0
2,CST_109330,Umesh Desai,37,M,N,Y,0.0,230153.17,204.0,Laborers,2.0,0.0,48416.60,41598.36,43,650,0,0,0
3,CST_128288,Rie,39,F,N,Y,0.0,122325.82,11941.0,Core staff,2.0,0.0,22574.36,32627.76,20,754,0,0,0
4,CST_151355,McCool,46,M,Y,Y,0.0,387286.00,1459.0,Core staff,1.0,0.0,38282.95,52950.64,75,927,0,0,0


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45528 entries, 0 to 45527
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id              45528 non-null  object 
 1   name                     45528 non-null  object 
 2   age                      45528 non-null  int64  
 3   gender                   45528 non-null  object 
 4   owns_car                 44981 non-null  object 
 5   owns_house               45528 non-null  object 
 6   no_of_children           44754 non-null  float64
 7   net_yearly_income        45528 non-null  float64
 8   no_of_days_employed      45065 non-null  float64
 9   occupation_type          45528 non-null  object 
 10  total_family_members     45445 non-null  float64
 11  migrant_worker           45441 non-null  float64
 12  yearly_debt_payments     45433 non-null  float64
 13  credit_limit             45528 non-null  float64
 14  credit_limit_used(%)  

In [55]:
df.drop(["name"],axis=1,inplace=True)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45528 entries, 0 to 45527
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id              45528 non-null  object 
 1   age                      45528 non-null  int64  
 2   gender                   45528 non-null  object 
 3   owns_car                 44981 non-null  object 
 4   owns_house               45528 non-null  object 
 5   no_of_children           44754 non-null  float64
 6   net_yearly_income        45528 non-null  float64
 7   no_of_days_employed      45065 non-null  float64
 8   occupation_type          45528 non-null  object 
 9   total_family_members     45445 non-null  float64
 10  migrant_worker           45441 non-null  float64
 11  yearly_debt_payments     45433 non-null  float64
 12  credit_limit             45528 non-null  float64
 13  credit_limit_used(%)     45528 non-null  int64  
 14  credit_score          

In [57]:
df["no_of_children"].unique()

array([ 0.,  1.,  2., nan,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

In [58]:
df["owns_car"].unique()

array(['N', 'Y', nan], dtype=object)

In [59]:
df["owns_house"].unique()

array(['Y', 'N'], dtype=object)

In [60]:
df["occupation_type"].unique()

array(['Unknown', 'Laborers', 'Core staff', 'Accountants',
       'High skill tech staff', 'Sales staff', 'Managers', 'Drivers',
       'Medicine staff', 'Cleaning staff', 'HR staff', 'Security staff',
       'Cooking staff', 'Waiters/barmen staff', 'Low-skill Laborers',
       'Private service staff', 'Secretaries', 'Realty agents',
       'IT staff'], dtype=object)

In [61]:
df["migrant_worker"].unique()
len(df.loc[df["migrant_worker"].isnull()])

87

In [62]:
df["yearly_debt_payments"].unique()

array([33070.28, 15329.53, 48416.6 , ..., 22707.51, 20150.1 , 34603.78])

In [63]:
df["credit_card_default"].unique()

array([1, 0], dtype=int64)

In [64]:
X=df.drop('credit_card_default',axis=1).copy()
X.head()

,customer_id,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months
0,CST_115179,46,F,N,Y,0.0,107934.04,612.0,Unknown,1.0,1.0,33070.28,18690.93,73,544,2,1
1,CST_121920,29,M,N,Y,0.0,109862.62,2771.0,Laborers,2.0,0.0,15329.53,37745.19,52,857,0,0
2,CST_109330,37,M,N,Y,0.0,230153.17,204.0,Laborers,2.0,0.0,48416.60,41598.36,43,650,0,0
3,CST_128288,39,F,N,Y,0.0,122325.82,11941.0,Core staff,2.0,0.0,22574.36,32627.76,20,754,0,0
4,CST_151355,46,M,Y,Y,0.0,387286.00,1459.0,Core staff,1.0,0.0,38282.95,52950.64,75,927,0,0


In [65]:
y=df["credit_card_default"]
y.head()

0    1
1    0
2    0
3    0
4    0
Name: credit_card_default, dtype: int64

In [66]:
X.dtypes

customer_id                 object
age                          int64
gender                      object
owns_car                    object
owns_house                  object
no_of_children             float64
net_yearly_income          float64
no_of_days_employed        float64
occupation_type             object
total_family_members       float64
migrant_worker             float64
yearly_debt_payments       float64
credit_limit               float64
credit_limit_used(%)         int64
credit_score                 int64
prev_defaults                int64
default_in_last_6months      int64
dtype: object

In [67]:
X.drop(["customer_id","no_of_children"],axis=1,inplace=True)

In [68]:
X.head()

,age,gender,owns_car,owns_house,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months
0,46,F,N,Y,107934.04,612.0,Unknown,1.0,1.0,33070.28,18690.93,73,544,2,1
1,29,M,N,Y,109862.62,2771.0,Laborers,2.0,0.0,15329.53,37745.19,52,857,0,0
2,37,M,N,Y,230153.17,204.0,Laborers,2.0,0.0,48416.60,41598.36,43,650,0,0
3,39,F,N,Y,122325.82,11941.0,Core staff,2.0,0.0,22574.36,32627.76,20,754,0,0
4,46,M,Y,Y,387286.00,1459.0,Core staff,1.0,0.0,38282.95,52950.64,75,927,0,0


In [69]:
X.dtypes

age                          int64
gender                      object
owns_car                    object
owns_house                  object
net_yearly_income          float64
no_of_days_employed        float64
occupation_type             object
total_family_members       float64
migrant_worker             float64
yearly_debt_payments       float64
credit_limit               float64
credit_limit_used(%)         int64
credit_score                 int64
prev_defaults                int64
default_in_last_6months      int64
dtype: object

In [70]:
X_encoded=pd.get_dummies(X,columns=["gender","owns_car","owns_house","occupation_type"])

In [71]:
X_encoded.dtypes

age                                        int64
net_yearly_income                        float64
no_of_days_employed                      float64
total_family_members                     float64
migrant_worker                           float64
yearly_debt_payments                     float64
credit_limit                             float64
credit_limit_used(%)                       int64
credit_score                               int64
prev_defaults                              int64
default_in_last_6months                    int64
gender_F                                   uint8
gender_M                                   uint8
gender_XNA                                 uint8
owns_car_N                                 uint8
owns_car_Y                                 uint8
owns_house_N                               uint8
owns_house_Y                               uint8
occupation_type_Accountants                uint8
occupation_type_Cleaning staff             uint8
occupation_type_Cook

In [72]:
X_encoded.loc[:,"age":"default_in_last_6months"]=X_encoded.loc[:,"age":"default_in_last_6months"].fillna(method='ffill').fillna('bfill')

In [73]:
X_encoded.dtypes

age                                        int64
net_yearly_income                        float64
no_of_days_employed                      float64
total_family_members                     float64
migrant_worker                           float64
yearly_debt_payments                     float64
credit_limit                             float64
credit_limit_used(%)                       int64
credit_score                               int64
prev_defaults                              int64
default_in_last_6months                    int64
gender_F                                   uint8
gender_M                                   uint8
gender_XNA                                 uint8
owns_car_N                                 uint8
owns_car_Y                                 uint8
owns_house_N                               uint8
owns_house_Y                               uint8
occupation_type_Accountants                uint8
occupation_type_Cleaning staff             uint8
occupation_type_Cook

In [74]:
sum(y)/len(y)

0.08120277631347741

In [75]:
X_train,X_test,y_train,y_test=train_test_split(X_encoded,y,random_state=42,stratify=y)

In [76]:
sum(y_train)/len(y_train)

0.08121009781526387

In [77]:
sum(y_test)/len(y_test)

0.08118081180811808

In [78]:
clf_xgb=xgb.XGBClassifier(objective="binary:logistic",missing=1,seed=28)
clf_xgb.fit(X_train,y_train,verbose=True,early_stopping_rounds=10,eval_metric='aucpr',eval_set=[(X_test,y_test)])

[0]	validation_0-aucpr:0.95807
[1]	validation_0-aucpr:0.95985
[2]	validation_0-aucpr:0.96111


C:\Users\Mrudhulraj Natarajan\.conda\envs\NLP\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[3]	validation_0-aucpr:0.96210
[4]	validation_0-aucpr:0.96132
[5]	validation_0-aucpr:0.96144
[6]	validation_0-aucpr:0.96121
[7]	validation_0-aucpr:0.96121
[8]	validation_0-aucpr:0.96090
[9]	validation_0-aucpr:0.96140
[10]	validation_0-aucpr:0.96156
[11]	validation_0-aucpr:0.96133
[12]	validation_0-aucpr:0.96140
[13]	validation_0-aucpr:0.96161


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=1,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=28,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=28,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [31]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
bootstrap = [True, False]
max_features = ['auto','sqrt']
min_samples_leaf = [1,2,4]
min_samples_split = [2,5,10]
n_estimators = [50,100,150,200]
max_depth = [4,6,10,12, None]

# Specify "parameter distributions" rather than a "parameter grid"
param_dist = dict(n_estimators=n_estimators, max_depth = max_depth, bootstrap=bootstrap,
                  min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split,
                  max_features=max_features)

# n_iter controls the number of searches

rand = RandomizedSearchCV(RandomForestClassifier(), param_dist, cv=10, scoring='accuracy',
                          n_iter=10, random_state=42)
rand.fit(X_train, y_train)

RandomizedSearchCV(cv=10, estimator=RandomForestClassifier(),
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [4, 6, 10, 12, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 150, 200]},
                   random_state=42, scoring='accuracy')

In [32]:
def get_best_model(model_grid):
    best_k = model_grid.cv_results_['rank_test_score'][0]
    print (f"Mean score: {model_grid.cv_results_['mean_test_score'][best_k]}",
           f"Std: {model_grid.cv_results_['std_test_score'][best_k]}")
           
# Examine the best model
print(rand.best_score_)
print(rand.best_params_)
get_best_model(rand)

0.9801147544217634
{'n_estimators': 150, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': None, 'bootstrap': True}
Mean score: 0.9785626148809357 Std: 0.0020075148179219855


In [42]:

rand_final=RandomForestClassifier(bootstrap = True,max_features = 'sqrt', random_state=42,max_depth = 10,n_estimators = 150,min_samples_split = 5,min_samples_leaf = 2)

In [43]:
rand_final.fit(X_train,y_train)

RandomForestClassifier(max_depth=10, max_features='sqrt', min_samples_leaf=2,
                       min_samples_split=5, n_estimators=150, random_state=42)

In [31]:
test=pd.read_csv("test.csv")

In [32]:
test.drop(["name","customer_id","no_of_children"],axis=1,inplace=True)

In [33]:
test=pd.get_dummies(test,columns=["gender","owns_car","owns_house","occupation_type"])

In [34]:
test.dtypes

age                                        int64
net_yearly_income                        float64
no_of_days_employed                      float64
total_family_members                     float64
migrant_worker                           float64
yearly_debt_payments                     float64
credit_limit                             float64
credit_limit_used(%)                       int64
credit_score                               int64
prev_defaults                              int64
default_in_last_6months                    int64
gender_F                                   uint8
gender_M                                   uint8
gender_XNA                                 uint8
owns_car_N                                 uint8
owns_car_Y                                 uint8
owns_house_N                               uint8
owns_house_Y                               uint8
occupation_type_Accountants                uint8
occupation_type_Cleaning staff             uint8
occupation_type_Cook

In [35]:
test.loc[:,"age":"default_in_last_6months"]=test.loc[:,"age":"default_in_last_6months"].fillna(method='ffill')

In [36]:
test.shape

(11383, 37)

In [37]:
y_pred=clf_xgb.predict(test)

In [38]:
y_pred

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [39]:
len(y_pred)

11383

In [40]:
final=pd.read_csv("test.csv")

In [41]:
final

,customer_id,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months
0,CST_142525,Siva,52,F,Y,N,0.0,232640.53,998.0,Unknown,2.0,0.0,14406.73,26524.40,4,779,0,0
1,CST_129215,Scott,48,F,N,N,1.0,284396.79,1338.0,Unknown,3.0,0.0,57479.99,68998.72,70,806,0,0
2,CST_138443,Victoria,50,F,N,N,1.0,149419.28,1210.0,Unknown,3.0,0.0,21611.01,25187.80,71,528,2,0
3,CST_123812,John McCrank,30,F,N,N,1.0,160437.54,503.0,Laborers,2.0,1.0,28990.76,29179.39,9,815,0,0
4,CST_144450,Martinne,52,M,N,Y,0.0,233480.37,157.0,Laborers,2.0,1.0,54213.72,82331.82,82,613,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11378,CST_142412,Solarina,53,F,N,N,0.0,266824.38,3051.0,Sales staff,1.0,0.0,17791.56,64483.93,74,823,0,0
11379,CST_107967,Jonathan Cable,33,F,NaN,N,0.0,124310.85,365248.0,Unknown,1.0,0.0,26576.79,29851.73,60,712,0,0
11380,CST_134002,Robin,27,M,Y,Y,1.0,364652.81,3431.0,Laborers,3.0,0.0,28658.55,69336.96,67,674,0,0
11381,CST_146856,Lauren,36,F,N,Y,0.0,128769.02,16320.0,Medicine staff,1.0,0.0,11527.38,26908.03,17,684,0,0


In [42]:
y_pred=list(y_pred)

In [43]:
final["credit_card_default"]=y_pred

In [44]:
final

,customer_id,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
0,CST_142525,Siva,52,F,Y,N,0.0,232640.53,998.0,Unknown,2.0,0.0,14406.73,26524.40,4,779,0,0,0
1,CST_129215,Scott,48,F,N,N,1.0,284396.79,1338.0,Unknown,3.0,0.0,57479.99,68998.72,70,806,0,0,0
2,CST_138443,Victoria,50,F,N,N,1.0,149419.28,1210.0,Unknown,3.0,0.0,21611.01,25187.80,71,528,2,0,1
3,CST_123812,John McCrank,30,F,N,N,1.0,160437.54,503.0,Laborers,2.0,1.0,28990.76,29179.39,9,815,0,0,0
4,CST_144450,Martinne,52,M,N,Y,0.0,233480.37,157.0,Laborers,2.0,1.0,54213.72,82331.82,82,613,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11378,CST_142412,Solarina,53,F,N,N,0.0,266824.38,3051.0,Sales staff,1.0,0.0,17791.56,64483.93,74,823,0,0,0
11379,CST_107967,Jonathan Cable,33,F,NaN,N,0.0,124310.85,365248.0,Unknown,1.0,0.0,26576.79,29851.73,60,712,0,0,0
11380,CST_134002,Robin,27,M,Y,Y,1.0,364652.81,3431.0,Laborers,3.0,0.0,28658.55,69336.96,67,674,0,0,0
11381,CST_146856,Lauren,36,F,N,Y,0.0,128769.02,16320.0,Medicine staff,1.0,0.0,11527.38,26908.03,17,684,0,0,0


In [45]:
final.dtypes

customer_id                 object
name                        object
age                          int64
gender                      object
owns_car                    object
owns_house                  object
no_of_children             float64
net_yearly_income          float64
no_of_days_employed        float64
occupation_type             object
total_family_members       float64
migrant_worker             float64
yearly_debt_payments       float64
credit_limit               float64
credit_limit_used(%)         int64
credit_score                 int64
prev_defaults                int64
default_in_last_6months      int64
credit_card_default          int64
dtype: object

In [46]:
final=final[["customer_id","credit_card_default"]]

In [47]:
final

,customer_id,credit_card_default
0,CST_142525,0
1,CST_129215,0
2,CST_138443,1
3,CST_123812,0
4,CST_144450,1
...,...,...
11378,CST_142412,0
11379,CST_107967,0
11380,CST_134002,0
11381,CST_146856,0


In [48]:
final.to_csv("Result_4.csv")